# 2. Análise Exploratória (EDA) e Modelagem Preditiva

**Objetivo:** Este notebook realiza a análise exploratória dos dados tratados e desenvolve um modelo de machine learning para prever o número de casos de violência.

**Passos:**
1. **Carregamento dos Dados:** Carregar o dataset tratado (`violencia_tratado.csv`).
2. **Análise Exploratória (EDA):**
   - Gerar gráficos de tendência.
   - Analisar correlações entre variáveis.
3. **Preparação para Modelagem:**
   - Selecionar features (variáveis preditoras) e target (variável alvo).
   - Dividir os dados em conjuntos de treino e teste.
4. **Treinamento e Avaliação:**
   - Treinar modelos de Regressão (Random Forest e XGBoost).
   - Avaliar os modelos usando métricas como MAE, RMSE e R².
5. **Seleção e Salvamento do Modelo:**
   - Escolher o modelo com melhor desempenho.
   - Salvar o modelo treinado no arquivo `model.pkl` para ser usado pela aplicação Flet.

## 1. Carregamento e Preparação dos Dados

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import os

# Define o tema padrão para os gráficos do Plotly
px.defaults.template = "plotly_dark"

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/macbookair/modelo-de-previsao-de-casos-de-violencia-no-brasil/venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <E8D72161-CCD1-3423-9388-36D4CA0A7524> /Users/macbookair/modelo-de-previsao-de-casos-de-violencia-no-brasil/venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no su

In [ ]:
# --- Geração de Dados Simulados ---
# Em um cenário real, você carregaria o CSV gerado no notebook 01.
# Como não temos o notebook 01, vamos criar um DataFrame simulado mais robusto aqui.

DATA_PATH = '../data/processed/violencia_tratado.csv'
MODEL_PATH = '../model.pkl'

def gerar_dados_simulados(path):
    print("Gerando dados simulados, pois o arquivo tratado não foi encontrado...")
    anos = range(2015, 2023)
    estados = ['SP', 'RJ', 'MG', 'BA']
    tipos_violencia = ['Doméstica', 'Sexual', 'Psicológica']
    faixas_etarias = ['10-19 anos', '20-29 anos', '30-39 anos']
    
    data = []
    for ano in anos:
        for estado in estados:
            for tipo in tipos_violencia:
                for faixa in faixas_etarias:
                    # Fatores base para simulação
                    base_casos = 100 + estados.index(estado) * 50 + tipos_violencia.index(tipo) * 20
                    tendencia_temporal = (ano - 2015) * 10
                    ruido = np.random.randint(-20, 20)
                    
                    # Simulação de indicadores socioeconômicos
                    idhm = 0.75 + (estados.index(estado) * 0.02) + (ano - 2015) * 0.005 + np.random.uniform(-0.01, 0.01)
                    taxa_desemprego = 12.0 - (estados.index(estado) * 0.5) - (ano - 2015) * 0.2 + np.random.uniform(-0.5, 0.5)
                    renda_media = 1200 + (estados.index(estado) * 100) + (ano - 2015) * 50 + np.random.uniform(-50, 50)
                    
                    # Fórmula para o número de casos
                    casos = base_casos + tendencia_temporal + ruido - (idhm * 50) + (taxa_desemprego * 5) - (renda_media * 0.01)
                    
                    data.append({
                        'ano': ano,
                        'estado': estado,
                        'tipo_violencia': tipo,
                        'faixa_etaria': faixa,
                        'casos': int(max(0, casos)),
                        'idhm': round(idhm, 3),
                        'taxa_desemprego': round(taxa_desemprego, 2),
                        'renda_media': int(renda_media)
                    })
    
    df = pd.DataFrame(data)
    # Garantir que o diretório exista
    os.makedirs(os.path.dirname(path), exist_ok=True)
    df.to_csv(path, index=False)
    return df

# Tenta carregar os dados, se não encontrar, gera dados simulados
try:
    df = pd.read_csv(DATA_PATH)
    print(f"Dados carregados de {DATA_PATH}")
except FileNotFoundError:
    df = gerar_dados_simulados(DATA_PATH)

df['ano'] = pd.to_datetime(df['ano'], format='%Y')
df.head()

## 2. Análise Exploratória de Dados (EDA)

### 2.1. Tendência de Casos por Ano (Total Brasil)

In [ ]:
casos_por_ano = df.groupby(df['ano'].dt.year)['casos'].sum().reset_index()

fig = px.line(casos_por_ano, x='ano', y='casos', title='Evolução do Total de Casos de Violência por Ano', 
              markers=True, labels={'ano': 'Ano', 'casos': 'Número Total de Casos'})
fig.show()

### 2.2. Distribuição de Casos por Estado e Tipo de Violência

In [ ]:
casos_agrupados = df.groupby(['estado', 'tipo_violencia'])['casos'].sum().reset_index()

fig = px.bar(casos_agrupados, x='estado', y='casos', color='tipo_violencia', 
             title='Total de Casos por Estado e Tipo de Violência', barmode='group',
             labels={'estado': 'Estado', 'casos': 'Número Total de Casos', 'tipo_violencia': 'Tipo de Violência'})
fig.show()

### 2.3. Correlação entre Variáveis

In [ ]:
numeric_df = df[['casos', 'idhm', 'taxa_desemprego', 'renda_media']]
correlation_matrix = numeric_df.corr()

fig = go.Figure(data=go.Heatmap(
                    z=correlation_matrix.values,
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    colorscale='RdBu_r', # Escala de cor (vermelho-azul)
                    zmin=-1, zmax=1, # Fixa a escala de -1 a 1
                    text=correlation_matrix.round(2).values,
                    texttemplate="%{text}"))

fig.update_layout(title='Mapa de Calor: Correlação entre Casos e Indicadores Socioeconômicos')
fig.show()

## 3. Preparação para Modelagem

In [ ]:
# O modelo irá prever o número de 'casos' com base nos indicadores socioeconômicos.
# As variáveis categóricas (estado, tipo, etc.) são usadas para filtrar os dados na aplicação,
# mas não serão usadas como features diretas neste modelo simplificado.

features = ['idhm', 'taxa_desemprego', 'renda_media']
target = 'casos'

X = df[features]
y = df[target]

# Dividir os dados em 80% para treino e 20% para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

## 4. Treinamento e Avaliação de Modelos

In [ ]:
# Dicionário para armazenar os modelos
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

# Dicionário para armazenar os resultados
results = {}

for name, model in models.items():
    # Treinar o modelo
    model.fit(X_train, y_train)
    
    # Fazer previsões no conjunto de teste
    y_pred = model.predict(X_test)
    
    # Calcular as métricas
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    # Armazenar os resultados
    results[name] = {'MAE': mae, 'RMSE': rmse, 'R²': r2}
    
    print(f"--- Resultados para {name} ---")
    print(f"MAE (Erro Médio Absoluto): {mae:.2f}")
    print(f"RMSE (Raiz do Erro Quadrático Médio): {rmse:.2f}")
    print(f"R² (Coeficiente de Determinação): {r2:.2f}")
    print("\n")

# Converter resultados para um DataFrame para fácil visualização
results_df = pd.DataFrame(results).T
results_df

## 5. Seleção e Salvamento do Melhor Modelo

In [ ]:
# Selecionar o melhor modelo com base no R² (quanto maior, melhor)
best_model_name = results_df['R²'].idxmax()
best_model = models[best_model_name]

print(f"O melhor modelo é: {best_model_name} com R² de {results_df.loc[best_model_name, 'R²']:.2f}")

# Salvar o modelo treinado em um arquivo .pkl
joblib.dump(best_model, MODEL_PATH)

print(f"Modelo salvo com sucesso em: {MODEL_PATH}")

### Teste Rápido do Modelo Carregado

In [ ]:
# Carregar o modelo para garantir que ele foi salvo corretamente
loaded_model = joblib.load(MODEL_PATH)

# Criar um exemplo de dados para previsão
exemplo_dados = pd.DataFrame({
    'idhm': [0.8],
    'taxa_desemprego': [10.5],
    'renda_media': [1800]
})

# Fazer uma previsão
previsao = loaded_model.predict(exemplo_dados)

print(f"Exemplo de dados para previsão:\n{exemplo_dados}\n")
print(f"Previsão de casos: {int(previsao[0])}")